In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import os
from torch_fidelity import calculate_metrics

In [2]:
# !pip install torch-fidelity
# !pip install --upgrade torch-fidelity
# !pip install pytorch-fid



In [9]:
import torch_fidelity
print(torch_fidelity.__version__)  # Expected: 0.3.0 or later
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


0.3.0


In [10]:
# Directory to save real images
real_images_path = './real_images/'
os.makedirs(real_images_path, exist_ok=True)

In [11]:

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

train_data = datasets.CIFAR10(root='./data02', train=True, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)


Files already downloaded and verified


In [ ]:
# num_images_to_save = 5000
# images_saved = 0
# for real_images, _ in train_loader:  # `_` ignores the labels
#     for i in range(real_images.size(0)):
#         # Stop once we've saved the desired number of images
#         if images_saved >= num_images_to_save:
#             break
        
#         # Save the image to the folder
#         save_image(real_images[i], f"{real_images_path}/real_{images_saved}.png")
#         images_saved += 1
    
#     if images_saved >= num_images_to_save:
#         break

# print(f"Saved {images_saved} real images to {real_images_path}")

Saved 5000 real images to ./real_images/


In [15]:
real_images_path = './real_images/'
generated_images_path='./generatedImagesDraft05(5epochsGen2lr0.0015)'
from pytorch_fid import fid_score
import torch
from torch_fidelity import calculate_metrics

fid_value = fid_score.calculate_fid_given_paths(
    paths=[real_images_path, generated_images_path],
    batch_size=50,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    dims=2048,
)
print(f"FID Score: {fid_value}")


# # Your existing FID calculation code
# metrics = calculate_metrics(
#     input1=real_images_path,
#     input2=generated_images_path,
#     cuda=False,  # `torch_fidelity` doesn't yet support MPS directly
#     isc=False,   # Inception Score not needed
#     fid=True     # Compute FID
# )
# print(f"FID Score: {metrics['frechet_inception_distance']}")



100%|██████████| 100/100 [00:14<00:00,  6.89it/s]


100%|██████████| 1/1 [00:02<00:00,  2.46s/it]


FID Score: 489.58442705699014


In [32]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
from sklearn.metrics import precision_score, recall_score, accuracy_score
import os
from PIL import Image
import numpy as np

In [33]:

# Load the pre-trained ResNet18 model
model = models.resnet18(pretrained=True)
model.eval().to(device)

c:\Users\poorna raavi\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\poorna raavi\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\poorna raavi/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 61.1MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [34]:
# Define the preprocessing transformations (for ResNet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ResNet standards
])

# Function to load and preprocess images
def load_images_from_folder(folder_path):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        if img_path.endswith(".png"):
            image = Image.open(img_path)
            image = transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to device
            images.append(image)
            labels.append(1 if "real" in filename else 0)  # Label real images as 1 and generated as 0
    return torch.cat(images), np.array(labels)


In [35]:
real_images, real_labels = load_images_from_folder('./real_images/')
generated_images, generated_labels = load_images_from_folder('./generatedImagesDraft04(500epochsGen10)/')

# Combine both datasets for evaluation
all_images = torch.cat((real_images, generated_images), dim=0)
all_labels = np.concatenate((real_labels, generated_labels), axis=0)


In [37]:
def classify_images(images):
    all_preds = []
    with torch.no_grad():
        for image in images:
            image = image.unsqueeze(0)  # Add batch dimension (1, channels, height, width)
            outputs = model(image)
            _, predicted_class = torch.max(outputs, 1)
            all_preds.append(predicted_class.item())
    return np.array(all_preds)


In [ ]:

# Get predictions for all images
predictions = classify_images(all_images)

In [ ]:
# Calculate Precision, Recall, Accuracy
precision = precision_score(all_labels, predictions)
recall = recall_score(all_labels, predictions)
accuracy = accuracy_score(all_labels, predictions)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")


NameError: name 'predictions' is not defined

In [ ]:

# # Set paths to your saved images
# real_images_path = './real_images/'  # Replace with your actual path to real images
# generated_images_path = './generatedImagesDraft04(500epochsGen10)/'  # Replace with your actual path to generated images


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(3, 128, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()  # Outputs a probability of real/fake
        )
    
    def forward(self, x):
        return self.disc(x)


In [ ]:

discriminator = Discriminator().to(device)